In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for month in ["January","February","March","April","May","June","July","August","September","October","November","December"]:
  s="/content/drive/MyDrive/2014_data/"+month+".zip"
  !unzip "$s" -d "/content/drive/MyDrive/2014_data"


In [ ]:
import os
import datetime
import glob
import pandas as pd
CE_2014=[]
PE_2014=[]

for month in ["January","February","March","April","May","June","July","August","September","October","November","December"]:
  wordir="/content/drive/MyDrive/2014_data/"+month
  os.environ['filename']=wordir
  os.chdir(os.environ['filename'])
  extension = "csv"
  Monthly_Call= [ i for i in glob.glob('CE*.{}'.format(extension))]
  Monthly_Put= [i for i in glob.glob('PE*.{}'.format(extension))]
  DF_CE=[]
  for f in Monthly_Call :
    df=pd.read_csv(f,header=None)
    if(len(df.columns)==9):
      df=df.iloc[:,:-1]
    df.columns = ["Strike Price","Date","Time","Open","High","Low","Close","Volume"]
    df.iloc[:,1]=pd.to_datetime(df.iloc[:,1],format='%d/%m/%Y')
    DF_CE.append(df)
  DF_PE=[]
  for f in Monthly_Put :
    df=pd.read_csv(f,header=None)
    if(len(df.columns)==9):
      df=df.iloc[:,:-1]
    df.columns = ["Strike Price","Date","Time","Open","High","Low","Close","Volume"]
    df.iloc[:,1]=pd.to_datetime(df.iloc[:,1],format='%d/%m/%Y')
    
    DF_PE.append(df)
  combined_csv_call= pd.concat(DF_CE,axis=0)
  combined_csv_put= pd.concat(DF_PE,axis=0)

  df_ce=combined_csv_call.iloc[:,:-1].set_index(["Date"])
  df_pe=combined_csv_put.iloc[:,:-1].set_index(["Date"])
  
  ohlc_dict = {        
    'Open':'first',
    'High':'max',                                                                                                                                                                                                            
    'Low':'min',                                                                                                        
    'Close': 'last',                                                                                            
    }
  df_ce=df_ce.groupby(["Strike Price"]).resample('1D', closed='left', label='left').apply(ohlc_dict)
  df_pe=df_pe.groupby(["Strike Price"]).resample('1D', closed='left', label='left').agg(ohlc_dict)
 
  name_CE = month+"-combinedCE_olhc.csv"
  name_PE=month+"-combinedPE_olhc.csv"
  os.chdir("/content/drive/MyDrive/2014_data")

  CE_2014.append(df_ce)
  PE_2014.append(df_pe)
  
 
PE_2014[0].head(100)

Open   High    Low  Close
Strike Price Date                                 
PE 5800      2013-12-12  26.2  26.20  21.30  24.60
             2013-12-13  29.0  30.35  25.10  28.70
             2013-12-14   NaN    NaN    NaN    NaN
             2013-12-15   NaN    NaN    NaN    NaN
             2013-12-16  30.0  36.00  27.70  29.55
...                       ...    ...    ...    ...
PE 5900      2014-01-28   4.0   6.00   1.30   1.45
PE 6000      2013-12-12  45.4  54.00  45.25  53.50
             2013-12-13  60.0  65.85  56.85  62.75
             2013-12-14   NaN    NaN    NaN    NaN
             2013-12-15   NaN    NaN    NaN    NaN

[100 rows x 4 columns]

In [ ]:
Date_thurs_last_month=pd.to_datetime(pd.Series(["30/01/2014","27/02/2014","27/03/2014","24/04/2014","29/05/2014","26/06/2014","31/07/2014","28/08/2014","25/09/2014","30/10/2014","27/11/2014","25/12/2014"]),format='%d/%m/%Y')
Date_thurs_last_month.head()


0   2014-01-30
1   2014-02-27
2   2014-03-27
3   2014-04-24
4   2014-05-29
dtype: datetime64[ns]

In [ ]:
CE_2014[0]

Open    High     Low   Close Expiration Day
Strike Price Date                                                     
CE 6200      2013-12-12  253.95  257.30  215.10  216.00        49 days
             2013-12-13  195.00  199.95  173.70  183.50        48 days
             2013-12-14     NaN     NaN     NaN     NaN        47 days
             2013-12-15     NaN     NaN     NaN     NaN        46 days
             2013-12-16  169.55  180.00  161.35  164.00        45 days
...                         ...     ...     ...     ...            ...
CE 6600      2014-01-20    2.00    2.55    1.20    1.85        10 days
             2014-01-21    6.05    6.05    1.05    1.45         9 days
             2014-01-22    1.80    2.00    1.00    1.35         8 days
             2014-01-23    1.30    1.35    0.85    1.00         7 days
             2014-01-24    0.95    1.40    0.60    0.65         6 days

[230 rows x 5 columns]

In [ ]:
for i in range(12):
  CE_2014[i]=CE_2014[i].reset_index()
  CE_2014[i]["Expiration Day"]= Date_thurs_last_month[i]-CE_2014[i].iloc[:,1]
  CE_2014[i]=CE_2014[i].set_index(["Strike Price","Date"])
  PE_2014[i]=PE_2014[i].reset_index()
  PE_2014[i]["Expiration Day"]= Date_thurs_last_month[i]-PE_2014[i].iloc[:,1]
  PE_2014[i]=PE_2014[i].set_index(["Strike Price","Date"])


In [ ]:
combined_csv_call= pd.concat(CE_2014,axis=0)
combined_csv_put= pd.concat(PE_2014,axis=0)
combined_csv_call["Expiration Day"]= pd.to_numeric(combined_csv_call["Expiration Day"].dt.days, downcast='integer')
combined_csv_put["Expiration Day"]= pd.to_numeric(combined_csv_put["Expiration Day"].dt.days, downcast='integer')
combined_csv_call.dropna(inplace=True)
combined_csv_put.dropna(inplace=True)

combined_csv_call.head()

Open    High     Low   Close  Expiration Day
Strike Price Date                                                      
CE 6200      2013-12-12  253.95  257.30  215.10  216.00              49
             2013-12-13  195.00  199.95  173.70  183.50              48
             2013-12-16  169.55  180.00  161.35  164.00              45
             2013-12-17  183.80  183.80  149.00  149.00              44
             2013-12-18  154.40  204.15  152.20  204.15              43

In [ ]:
combined_csv_call.to_csv("Call_2014_daily_exp_added.csv",encoding='utf-8-sig')
combined_csv_put.to_csv("Put_2014_daily_exp_added.csv",encoding='utf-8-sig')